In [61]:
''' Import MySQL connector '''
import mysql.connector as mysql

''' Set parameters for DB connection '''
db_user = 'root'
db_pwd = 'root'
db_host = '127.0.0.1'
db_name = 'set_cover'

''' Create connection with MySQL '''
cnx = mysql.connect(user=db_user, passwd=db_pwd,
                    host=db_host, db=db_name)                              

''' Create cursor to receive data '''
cursor = cnx.cursor()

''' Request demand table '''
query = 'SELECT * FROM demand;'
cursor.execute(query)
demand = cursor.fetchall()

'''Request tower table'''
query = 'SELECT * FROM tower;'
cursor.execute(query)
tower = cursor.fetchall()

''' Close cursor '''
cursor.close()
cnx.close()

In [73]:
import numpy as np
import random
import gurobipy as gpy

num_dem = len(demand)
num_tow = 1500
pct_thres = .02
done = False

while not done:
    tow_index = random.sample(range(0, len(tower)), num_tow) #random sample of specified number of towers
    tow_sm = [tower[i] for i in tow_index]
    
    close_towers = set() #make a set to have only unique tower values because we loop through multiple times 
    for tow in tow_sm:
        num_near = 0
        for dem in demand:
            dist = np.sqrt((dem[0] - tow[0])**2 + (dem[1] - tow[1])**2)
            if dist <= 100:
                num_near += 1
        if num_near >= pct_thres*num_dem: #consider this tower useful/close if there is a certain number of demands within 100 units 
            close_towers.add(tow)
    close_towers = list(close_towers)
    
    dem_tow = np.zeros((num_dem, len(close_towers))) #initialize matrix of demand locations and tower locations 
    
    for j, tow in enumerate(close_towers):
        for i, dem in enumerate(demand):
            dist = np.sqrt((dem[0] - tow[0])**2 + (dem[1] - tow[1])**2)
            if dist <= 100:
                dem_tow[i][j] = 1 #1 if tower is within feasible radius of demand location
    
    m = gpy.Model('CellTower')
    y = m.addMVar((len(close_towers), ), vtype = gpy.GRB.BINARY, name = 'y')
    m.update()
    
    m.setObjective(y.sum(), gpy.GRB.MINIMIZE)
    m.update()
    
    m.addConstrs(dem_tow @ y >= 1 for _ in range(1))
    m.update()
    
    m.setParam('MIPGap', 0.05)
    m.update()
    
    m.optimize()
    print(f'Gurobi Model Status: {m.Status}')
    
    if m.Status == 2: #stop when model is optimal
        done = True
    else:
        num_tow += 100 #if the model is infeasible, incease the number of towers considered

Set parameter MIPGap to value 0.05
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2000 rows, 1374 columns and 81489 nonzeros
Model fingerprint: 0x90a3cb2b
Variable types: 0 continuous, 1374 integer (1374 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 59.0000000
Presolve removed 707 rows and 181 columns
Presolve time: 0.29s
Presolved: 1293 rows, 1193 columns, 53230 nonzeros
Variable types: 0 continuous, 1193 integer (1193 binary)

Root relaxation: objective 3.785682e+01, 4000 iterations, 0.54 seconds (0.76 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Bes